In [44]:
!pip freeze > requirements-frozen.txt


I0000 00:00:1750647750.377580  840842 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Overview
The Model Context Protocol (MCP) is an open standard that simplifies how AI assistants connect with external data, tools, and systems. It achieves this by standardizing the way applications provide contextual information to Large Language Models (LLMs), creating a vital interface for models to interact directly with various external services.

Developers building MCP-enabled applications have the flexibility to utilize existing third-party MCP servers or implement their own custom server solutions.

This notebook focuses on the latter, demonstrating how to build custom MCP servers using Gemini 2.5 Pro. We will walk through code generation and testing for four specific examples:

#### MCP server code generation:
- Example 1: Building a BigQuery MCP Server
- Example 2: Building a MedlinePlus MCP Server
- Example 3: Building an NIH MCP Server
- Exmaple 4: Building a Cocktail MCP Server

#### MCP server code testing:
- Option 1: Use LangChain MCP Adaptor (working in Jupyter Notebook only, not Colab)
- Option 2: Build your own agent
- Option 3: Use Google ADK

## Get started

### Install Google Gen AI SDK and other required packages


In [2]:
%pip install --upgrade --quiet google-genai google-cloud-secret-manager mcp geopy black google-cloud-bigquery langchain-mcp-adapters langchain langchain-google-vertexai langgraph google-adk


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Import Libraries

In [4]:
import sys
import os
import asyncio
import datetime
import json
import requests
import black
import re

from typing import Union, Dict, List, Optional
from google.genai.types import (
    GenerateContentConfig,
)
from google import genai
from google.genai import types
from google.cloud import aiplatform
from typing import List, Dict, Any

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_google_vertexai import ChatVertexAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from IPython.display import display, Markdown
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage

### Helper function


In [5]:
def get_url_content(url):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)

        # Raise an exception if the request returned an error status code (like 404 or 500)
        response.raise_for_status()

        # Get the content of the response as text (HTML, in this case)
        # 'requests' automatically decodes the content based on HTTP headers
        file_content = response.text

        # Now you can work with the content
        print("Successfully fetched content")
        return file_content
        # Or, save it to a file:
        # with open("server_page.html", "w", encoding="utf-8") as f:
        #     f.write(file_content)
        # print("Content saved to server_page.html")

    except requests.exceptions.RequestException as e:
        # Handle potential errors during the request (e.g., network issues, DNS errors)
        print(f"Error fetching URL {url}: {e}")
    except requests.exceptions.HTTPError as e:
        # Handle HTTP error responses (e.g., 404 Not Found, 503 Service Unavailable)
        print(f"HTTP Error for {url}: {e}")


def format_python(raw_code, output_filename):

    try:
        # Format the code string using black
        # Use default FileMode which is generally recommended
        formatted_code = black.format_str(raw_code, mode=black.FileMode())

        # Save the formatted code to the specified file
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(formatted_code)

        print(f"Successfully formatted the code and saved it to '{output_filename}'")

    except black.InvalidInput as e:
        print(
            f"Error formatting code: The input string does not seem to be valid Python syntax."
        )
        print(f"Details: {e}")
    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


def extract_json_from_string(input_str: str) -> Optional[Union[Dict, List]]:
    """
    Extracts JSON data from a string, handling potential variations.

    This function attempts to find JSON data within a string. It specifically
    looks for JSON enclosed in Markdown-like code fences (```json ... ```).
    If such a block is found, it extracts and parses the content.
    If no code block is found, it attempts to parse the entire input string
    as JSON.

    Args:
        input_str: The string potentially containing JSON data. It might be
                   a plain JSON string or contain a Markdown code block
                   with JSON, possibly preceded by other text (like 'shame').

    Returns:
        The parsed JSON object (typically a dictionary or list) if valid
        JSON is found and successfully parsed.
        Returns None if no valid JSON is found, if parsing fails, or if the
        input is not a string.
    """
    if not isinstance(input_str, str):
        # Handle cases where input is not a string
        return None

    # Pattern to find JSON within ```json ... ``` blocks
    # - ````json`: Matches the start fence.
    # - `\s*`: Matches any leading whitespace after the fence marker.
    # - `(.*?)`: Captures the content (non-greedily) between the fences. This is group 1.
    # - `\s*`: Matches any trailing whitespace before the end fence.
    # - ` ``` `: Matches the end fence.
    # - `re.DOTALL`: Allows '.' to match newline characters.
    pattern = r"```json\s*(.*?)\s*```"
    match = re.search(pattern, input_str, re.DOTALL)

    json_string_to_parse = None

    if match:
        # If a markdown block is found, extract its content
        json_string_to_parse = match.group(
            1
        ).strip()  # Get captured group and remove surrounding whitespace
    else:
        # If no markdown block, assume the *entire* input might be JSON
        # We strip whitespace in case the string is just JSON with padding
        json_string_to_parse = input_str.strip()

    if not json_string_to_parse:
        # If after stripping, the potential JSON string is empty, return None
        return None

    try:
        # Attempt to parse the determined string (either from block or whole input)
        parsed_json = json.loads(json_string_to_parse)
        return parsed_json
    except json.JSONDecodeError:
        # Parsing failed, indicating the string wasn't valid JSON
        return None
    except Exception as e:
        # Catch other potential unexpected errors during parsing
        print(f"An unexpected error occurred during JSON parsing: {e}")
        return None


from pathlib import Path


def create_folder_if_not_exists(folder_path_str: str) -> bool:
    """
    Creates a folder (and any necessary parent folders) if it doesn't already exist.
    Uses print() for status and error messages.

    Args:
        folder_path_str (str): The path string for the folder to be created.
                               Can be relative or absolute.

    Returns:
        bool: True if the folder already exists or was successfully created,
              False if an error occurred during creation (e.g., permission denied).
    """
    try:
        # Convert the string path to a Path object
        folder_path = Path(folder_path_str)

        # Use mkdir() with options:
        # parents=True: Creates any necessary parent directories. Like 'mkdir -p'.
        # exist_ok=True: Doesn't raise an error if the directory already exists.
        folder_path.mkdir(parents=True, exist_ok=True)

        # Print confirmation (using resolve() to show the absolute path)
        print(f"Info: Successfully ensured folder exists: {folder_path.resolve()}")
        return True

    except PermissionError:
        print(
            f"Error: Permission denied: Could not create folder at '{folder_path_str}'."
        )
        return False
    except OSError as e:
        # Catch other OS-related errors (e.g., path is a file, invalid path format on Windows)
        print(f"Error: OS error creating folder '{folder_path_str}': {e}")
        return False
    except Exception as e:
        # Catch any other unexpected errors
        print(
            f"Error: An unexpected error occurred creating folder '{folder_path_str}': {e}"
        )
        return False

In [6]:
create_folder_if_not_exists("server")

Info: Successfully ensured folder exists: /home/jupyter/GenAI9/adk-mcp-tutorial/server


True

### Option 1 use a  Vertex AI project

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [7]:
import os
from IPython.display import JSON
import json

# Cloud project id.
PROJECT_IDS = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_IDS[0]  # @param {type:"string"}

if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API
# [your-project-id]

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

## Set up model id

In [9]:
MODEL_ID = "gemini-2.5-flash"

### Get system instruction context info

In [10]:
# The URL you want to fetch
url = "https://modelcontextprotocol.io/quickstart/server"
reference_content = get_url_content(url)

Successfully fetched content


### Set up system instruction

In [11]:
from pydantic import BaseModel


class ResponseSchema(BaseModel):
    python_code: str
    description: str


system_instruction = f"""
  You are an MCP server export.
  Your mission is to write python code for MCP server.
  Here's the MCP server development guide and example
  {reference_content}

"""

#### Set function to generate MCP server code

In [12]:
def generate_mcp_server(prompt):
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config=GenerateContentConfig(
            system_instruction=system_instruction,
            response_mime_type="application/json",
            response_schema=ResponseSchema,
        ),
    )

    return response.text

## Generate MCP Server Code

### Example 1:  Build MCP Server for Google Cloud BigQuery

In [13]:
prompt = """
  Please create an MCP server code for google cloud big query. It has two tools. One is to list tables for all datasets, the other is to describe a table. Google cloud project id and location will be provided in the query string. please use project id to access BigQuery client.
  Please output JSON output only.

"""

In [14]:
response_text = generate_mcp_server(prompt)

In [15]:
python_code = extract_json_from_string(response_text)["python_code"]
format_python(python_code, "server/bq.py")

Successfully formatted the code and saved it to 'server/bq.py'


### Example 2:  Build MCP server for Medlineplus website
Create an MCP server for
https://medlineplus.gov/about/developers/webservices/ API service

In [16]:
med_url = "https://medlineplus.gov/about/developers/webservices/"
med_prompt_base = """
  Please create an MCP server code for https://medlineplus.gov/about/developers/webservices/. It has one tool, get_medical_term. You provide a medical term, this tool will return explanation of the medial term.

  Here's the API details:

"""

prompt = [med_prompt_base, types.Part.from_uri(file_uri=med_url, mime_type="text/html")]
response_text = generate_mcp_server(prompt)
python_code = extract_json_from_string(response_text)["python_code"]

format_python(python_code, "server/med.py")

Successfully formatted the code and saved it to 'server/med.py'


### Example 3: Build MCP Server for NIH

In [ ]:
nih_url = "https://clinicaltables.nlm.nih.gov/apidoc/icd10cm/v3/doc.html"
nih_prompt_base = """
  Please create an MCP server code for NIH. It has one tool, get_icd_10_code. You provide a name or code, it will return top 5 results.

  Here's the API details:

"""
prompt = [nih_prompt_base, types.Part.from_uri(file_uri=nih_url, mime_type="text/html")]
response_text = generate_mcp_server(prompt)
python_code = extract_json_from_string(response_text)["python_code"]

format_python(python_code, "server/nih.py")

### Example 4: Build MCP Server for the Cocktail DB

In [18]:
ct_url = "https://www.thecocktaildb.com/api.php"
ct_prompt_base = """
  Please create an MCP server code for the cocktail db. It has 5 tools:
  1. search cocktail by name
  2. list all cocktail by first letter
  3. search ingredient by name.
  4. list random cocktails
  5. lookup full cocktail details by id

  Here's the API details:

"""
prompt = [ct_prompt_base, types.Part.from_uri(file_uri=ct_url, mime_type="text/html")]
response_text = generate_mcp_server(prompt)
python_code = extract_json_from_string(response_text)["python_code"]

format_python(python_code, "server/cocktail.py")

Successfully formatted the code and saved it to 'server/cocktail.py'


## Testing MCP Servers

### Option 1: Use LangChain MCP adaptor to test MCP servers
It works in Jupyter Notebook only, not working in Colab.

In [19]:
aiplatform.init(
    project=PROJECT_ID,
    location=LOCATION,
)

In [20]:
llm = ChatVertexAI(
    model="gemini-2.5-pro",
    temperature=0,
    max_tokens=None,
    max_retries=6,
    stop=None,
)

In [21]:
server_configs = {
    "nih": {
        "command": "python",
        "args": ["./server/nih.py"],
        "transport": "stdio",
    },
    "med": {
        "command": "python",
        "args": ["./server/med.py"],
        "transport": "stdio",
    },
    "bq": {
        "command": "python",
        "args": ["./server/bq.py"],
        "transport": "stdio",
    },
    "cocktail": {
        "command": "python",
        "args": ["./server/cocktail.py"],
        "transport": "stdio",
    },
}

Set up MCP Client using LangChain MCP Adaptor

In [22]:
# async def run_lc_react_agent(server_configs, message):
#     async with MultiServerMCPClient(server_configs) as client:
#         agent = create_react_agent(llm, client.get_tools())

#         agent_response = await agent.ainvoke({"messages": message})
#         for response in agent_response["messages"]:
#             user = ""

#             if isinstance(response, HumanMessage):
#                 user = "[User]"
#             elif isinstance(response, ToolMessage):
#                 user = "-Tool-"
#             elif isinstance(response, AIMessage):
#                 user = "[Agent]"

#             if isinstance(response.content, list):
#                 display(Markdown(f'{user}: {response.content[0].get("text", "")}'))
#                 continue
#             display(Markdown(f"{user}: {response.content}"))

In [23]:
async def run_lc_react_agent(server_configs, message):
    client = MultiServerMCPClient(server_configs)
    tools = await client.get_tools()
    agent = create_react_agent(llm, tools)
    
    agent_response = await agent.ainvoke({"messages": message})
    
    for response in agent_response["messages"]:
        user = ""
        if isinstance(response, HumanMessage):
            user = "[User]"
        elif isinstance(response, ToolMessage):
            user = "-Tool-"
        elif isinstance(response, AIMessage):
            user = "[Agent]"
        if isinstance(response.content, list):
            display(Markdown(f'{user}: {response.content[0].get("text", "")}'))
            continue
        display(Markdown(f"{user}: {response.content}"))

#### Example 1: BigQuery MCP server testing

In [24]:
await run_lc_react_agent(
    server_configs,
    "Please list my bigquery tables for project 'my-project-0004-346516', the location is 'us', yes give me all the tables list",
)

[User]: Please list my bigquery tables for project 'my-project-0004-346516', the location is 'us', yes give me all the tables list

[Agent]: 

-Tool-: Dataset: census
  - Table: census_data
Dataset: copy_data_analyticsshareddata
  - Table: testtable
Dataset: darryl_video_rag_demo_lh
  - Table: video_embeddings
Dataset: data_analytics_shared_data
  - Table: biglake_caspian_rls
  - Table: bigsearch_log_5b_5t_json_hourly
Dataset: formagent00
  - Table: timeseries_data
Dataset: gemini_bq_fn
  - Table: books
Dataset: model_deployment_monitoring_1911931973446664192
  - Table: serving_predict
Dataset: model_deployment_monitoring_7861011159342645248
  - Table: serving_predict
Dataset: movielens
  - Table: movies
  - Table: products
  - Table: ratings
  - Table: user_events
Dataset: movielens_gcp_sa_retail
  No tables found in this dataset.
Dataset: my_project_0004_346516_unique
  No tables found in this dataset.
Dataset: vertex_feature_transform_engine_staging_us
  No tables found in this dataset.
Dataset: visionai_dataset
  - Table: traffic-app

[Agent]: Of course, here is the list of tables for project 'my-project-0004-346516' in the 'us' location:

**Dataset: census**
* Table: census_data

**Dataset: copy_data_analyticsshareddata**
* Table: testtable

**Dataset: darryl_video_rag_demo_lh**
* Table: video_embeddings

**Dataset: data_analytics_shared_data**
* Table: biglake_caspian_rls
* Table: bigsearch_log_5b_5t_json_hourly

**Dataset: formagent00**
* Table: timeseries_data

**Dataset: gemini_bq_fn**
* Table: books

**Dataset: model_deployment_monitoring_1911931973446664192**
* Table: serving_predict

**Dataset: model_deployment_monitoring_7861011159342645248**
* Table: serving_predict

**Dataset: movielens**
* Table: movies
* Table: products
* Table: ratings
* Table: user_events

**Dataset: visionai_dataset**
* Table: traffic-app

The following datasets were found but contain no tables:
* movielens_gcp_sa_retail
* my_project_0004_346516_unique
* vertex_feature_transform_engine_staging_us

#### Example 2: MedlinePlus MCP server testing

In [25]:
await run_lc_react_agent(server_configs, "Please explain flu / Influenza in details")

[User]: Please explain flu / Influenza in details

[Agent]: 

-Tool-: What is the <span class="qt0"><span class="qt1">flu</span></span>?<p>The <span class="qt0"><span class="qt1">flu</span></span>, also called <span class="qt0"><span class="qt1">influenza</span></span>, is a respiratory infection caused by viruses. Each year, millions of Americans get sick with the <span class="qt0"><span class="qt1">flu</span></span>. Sometimes it causes mild illness. But it can also be serious or even deadly, especially for people over 65, newborn babies, and people with certain chronic illnesses.</p>What causes the <span class="qt0"><span class="qt1">flu</span></span>?<p>The <span class="qt0"><span class="qt1">flu</span></span> is caused by <span class="qt0"><span class="qt1">flu</span></span> viruses that spread from person to person. When someone with the <span class="qt0"><span class="qt1">flu</span></span> coughs, sneezes, or talks, they spray tiny droplets. These droplets can land in the mouths or noses of people who are nearby. Less often, a person may get <span class="qt0"><span class="qt1">flu</span></span> by touching a surface or object that has <span class="qt0"><span class="qt1">flu</span></span> virus on it and then touching their own mouth, nose, or possibly their eyes.</p>What are the symptoms of the <span class="qt0"><span class="qt1">flu</span></span>?<p>Symptoms of the <span class="qt0"><span class="qt1">flu</span></span> come on suddenly and may include:</p><ul><li>Fever or feeling feverish/chills</li><li>Cough</li><li>Sore throat</li><li>Runny or stuffy nose</li><li>Muscle or body aches</li><li>Headaches</li><li>Fatigue (tiredness)</li></ul><p>Some people may also have vomiting and diarrhea. This is more common in children.</p><p>Sometimes people have trouble figuring out whether they have a cold or the <span class="qt0"><span class="qt1">flu</span></span>. There are differences between them:</p>Signs and SymptomsColdFluStart of symptomsSlowlySuddenlyFeverRarelyUsuallyAchesSometimes (slight)UsuallyFatigue, weaknessSometimesUsuallyHeadacheRarelyCommonStuffy nose, sneezing, or sore throatCommonSometimes<p>Sometimes people say that they have a <span class="qt0"><span class="qt1">"flu"</span></span> when they really have something else. For example, <span class="qt0"><span class="qt1">"stomach flu"</span></span> isn't the <span class="qt0"><span class="qt1">flu</span></span>; it's gastroenteritis.</p>What other problems can the <span class="qt0"><span class="qt1">flu</span></span> cause?<p>Some people who get the <span class="qt0"><span class="qt1">flu</span></span> will develop complications. Some of these complications can be serious or even life-threatening. They include:</p><ul><li>Bronchitis</li><li>Ear infection</li><li>Sinus infection</li><li>Pneumonia</li><li>Inflammation of the heart (myocarditis), brain (encephalitis), or muscle tissues (myositis, rhabdomyolysis)</li></ul><p>The <span class="qt0"><span class="qt1">flu</span></span> also can make chronic health problems worse. For example, people with asthma may have asthma attacks while they have <span class="qt0"><span class="qt1">flu</span></span>.</p><p>Certain people are more likely to have complications from the <span class="qt0"><span class="qt1">flu</span></span>, including:</p><ul><li>Adults 65 and older</li><li>Pregnant women</li><li>Children younger than 5</li><li>People with certain chronic health conditions, such as asthma, diabetes, and heart disease</li></ul>How is the <span class="qt0"><span class="qt1">flu</span></span> diagnosed?<p>To diagnose the <span class="qt0"><span class="qt1">flu</span></span>, health care providers will first do a medical history and ask about your symptoms. There are several tests for the <span class="qt0"><span class="qt1">flu</span></span>. For the tests, your provider will swipe the inside of your nose or the back of your throat with a swab. Then the swab will be tested for the <span class="qt0"><span class="qt1">flu</span></span> virus.</p><p>Some tests are quick and give results in 15-20 minutes. But these tests are not as accurate as other <span class="qt0"><span class="qt1">flu</span></span> tests. These other tests can give you the results in one hour or several hours.</p>What are the treatments for the <span class="qt0"><span class="qt1">flu</span></span>?<p>Most people with the <span class="qt0"><span class="qt1">flu</span></span> recover on their own without medical care. People with mild cases of the <span class="qt0"><span class="qt1">flu</span></span> should stay home and avoid contact with others, except to get medical care.</p><p>But if you have symptoms of <span class="qt0"><span class="qt1">flu</span></span> and are in a high risk group or are very sick or worried about your illness, contact your health care provider. You might need antiviral medicines to treat your <span class="qt0"><span class="qt1">flu</span></span>. Antiviral medicines can make the illness milder and shorten the time you are sick. They also can prevent serious <span class="qt0"><span class="qt1">flu</span></span> complications. They usually work best when you start taking them within 2 days of getting sick.</p>Can the <span class="qt0"><span class="qt1">flu</span></span> be prevented?<p>The best way to prevent the <span class="qt0"><span class="qt1">flu is to get a flu</span></span> vaccine every year. But it's also important to have good health habits like covering your cough and washing your hands often. This can help stop the spread of germs and prevent the <span class="qt0"><span class="qt1">flu</span></span>.</p><p>Centers for Disease Control and Prevention</p>

[Agent]: The flu, also known as influenza, is a respiratory infection caused by viruses. It's a common illness that affects millions of people each year. While it can be mild for some, it can also be serious and even life-threatening, particularly for vulnerable populations like the elderly, young children, and individuals with chronic health conditions.

**Causes:**

The flu is caused by influenza viruses that spread from person to person through respiratory droplets released when an infected person coughs, sneezes, or talks. You can also get the flu by touching a surface contaminated with the virus and then touching your mouth, nose, or eyes.

**Symptoms:**

Flu symptoms tend to come on suddenly and can include:

*   Fever or feeling feverish/chills
*   Cough
*   Sore throat
*   Runny or stuffy nose
*   Muscle or body aches
*   Headaches
*   Fatigue (tiredness)
*   Vomiting and diarrhea (more common in children)

**Complications:**

In some cases, the flu can lead to serious complications, such as:

*   Bronchitis
*   Ear and sinus infections
*   Pneumonia
*   Inflammation of the heart (myocarditis), brain (encephalitis), or muscle tissues (myositis, rhabdomyolysis)

The flu can also worsen chronic health problems. For example, people with asthma may experience asthma attacks.

**Diagnosis:**

To diagnose the flu, healthcare providers will assess your symptoms and may use a flu test, which involves swabbing the inside of your nose or the back of your throat.

**Treatment:**

Most people with the flu recover on their own with rest and fluids. However, antiviral medications may be prescribed to those who are at high risk of complications or have a severe illness. These medications can help lessen the severity and duration of the illness.

**Prevention:**

The most effective way to prevent the flu is to get a flu vaccine each year. Good hygiene practices, such as frequent handwashing and covering your coughs and sneezes, are also important in preventing the spread of the virus.

#### Example 3: NIH MCP Server testing

In [26]:
await run_lc_react_agent(server_configs, "can you tell me icd-10 code for influenza A?")

[User]: can you tell me icd-10 code for influenza A?

[Agent]: 

-Tool-: No ICD-10 codes found for 'influenza A'.

[Agent]: I am sorry, I cannot find the ICD-10 code for "influenza A". I will try searching for "influenza" instead.

-Tool-: No ICD-10 codes found for 'influenza'.

[Agent]: I am sorry, I cannot find the ICD-10 code for "influenza" either. I will try searching for "flu" instead.

-Tool-: No ICD-10 codes found for 'flu'.

[Agent]: I am sorry, I am unable to find the ICD-10 code for influenza A at the moment. Please try again later.

### Example 4: Cocktail MCP server testing

In [27]:
await run_lc_react_agent(server_configs, "Please get full detail of cocktail margarita")

[User]: Please get full detail of cocktail margarita

[Agent]: 

-Tool-: ID: 11007, Name: Margarita, Category: Ordinary Drink
---
ID: 11118, Name: Blue Margarita, Category: Ordinary Drink
---
ID: 17216, Name: Tommy's Margarita, Category: Ordinary Drink
---
ID: 16158, Name: Whitecap Margarita, Category: Other / Unknown
---
ID: 12322, Name: Strawberry Margarita, Category: Ordinary Drink
---
ID: 178332, Name: Smashed Watermelon Margarita, Category: Cocktail

[Agent]: Which margarita are you looking for? I have found a few options: Margarita, Blue Margarita, Tommy's Margarita, Whitecap Margarita, Strawberry Margarita, Smashed Watermelon Margarita.

#### Agent streaming

In [ ]:
async def run_streaming_agent():
    client = MultiServerMCPClient(
        {
            "nih": {
                "command": "python",
                "args": ["./server/nih.py"],
                "transport": "stdio",
            },
        }
    )
    
    tools = await client.get_tools()
    agent = create_react_agent(llm, tools)
    
    # Initialize conversation history using simple tuples
    inputs = {"messages": []}
    print("Agent is ready. Type 'exit' to quit.")
    
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Exiting chat.")
            break
            
        # Append user message to history
        inputs["messages"].append(("user", user_input))
        
        # call our graph with streaming to see the steps
        async for state in agent.astream(inputs, stream_mode="values"):
            last_message = state["messages"][-1]
            print(last_message)
            last_message.pretty_print()
            
        # update the inputs with the agent's response (fix: use = instead of ==)
        inputs["messages"] = state["messages"]



In [28]:
# async def run_streaming_agent():
#     async with MultiServerMCPClient(
#         {
#             "nih": {
#                 "command": "python",
#                 "args": ["./server/nih.py"],
#                 "transport": "stdio",
#             },
#         }
#     ) as client:
#         agent = create_react_agent(llm, client.get_tools())
#         # Initialize conversation history using simple tuples
#         inputs = {"messages": []}

#         print("Agent is ready. Type 'exit' to quit.")
#         while True:
#             user_input = input("You: ")
#             if user_input.lower() == "exit":
#                 print("Exiting chat.")
#                 break

#             # Append user message to history
#             inputs["messages"].append(("user", user_input))

#             # call our graph with streaming to see the steps
#             async for state in agent.astream(inputs, stream_mode="values"):
#                 last_message = state["messages"][-1]
#                 print(last_message)
#                 last_message.pretty_print()

#             # update the inputs with the agent's response
#             inputs["messages"] == state["messages"]

In [ ]:
# await run_streaming_agent()

### Option 2: Build your own agent to test MCP servers


##### Gemini Agent

Within an MCP client session, this agent loop runs a multi-turn conversation loop with a Gemini model, handling tool calls via MCP server.

This function orchestrates the interaction between a user prompt, a Gemini model capable of function calling, and a session object that provides and executes tools. It handles the cycle of:
-  Gemini gets tool information from MCP client session
-  Sending the user prompt (and conversation history) to the model.
-  If the model requests tool calls, Gemini makes initial function calls to get structured data as per schema, and
-  Sending the tool execution results back to the model.
-  Repeating until the model provides a text response or the maximum number of tool execution turns is reached.
-  Gemini generates final response based on tool responses and original query.
  
MCP integration with Gemini

<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/mcp/mcp_tool_call.png" alt="MCP with Gemini" height="700">

In [33]:
# --- Configuration ---
# Consider using a more recent/recommended model if available and suitable

DEFAULT_MAX_TOOL_TURNS = 5  # Maximum consecutive turns for tool execution
DEFAULT_INITIAL_TEMPERATURE = (
    0.0  # Temperature for the first LLM call (more deterministic)
)
DEFAULT_TOOL_CALL_TEMPERATURE = (
    1.0  # Temperature for LLM calls after tool use (potentially more creative)
)

# Make tool calls via MCP Server


async def _execute_tool_calls(
    function_calls: List[types.FunctionCall], session: ClientSession
) -> List[types.Part]:
    """
    Executes a list of function calls requested by the Gemini model via the session.

    Args:
        function_calls: A list of FunctionCall objects from the model's response.
        session: The session object capable of executing tools via `call_tool`.

    Returns:
        A list of Part objects, each containing a FunctionResponse corresponding
        to the execution result of a requested tool call.
    """
    tool_response_parts: List[types.Part] = []
    print(f"--- Executing {len(function_calls)} tool call(s) ---")

    for func_call in function_calls:
        tool_name = func_call.name
        # Ensure args is a dictionary, even if missing or not a dict type
        args = func_call.args if isinstance(func_call.args, dict) else {}
        print(f"  Attempting to call session tool: '{tool_name}' with args: {args}")

        tool_result_payload: Dict[str, Any]
        try:
            # Execute the tool using the provided session object
            # Assumes session.call_tool returns an object with attributes
            # like `isError` (bool) and `content` (list of Part-like objects).
            tool_result = await session.call_tool(tool_name, args)
            print(f"  Session tool '{tool_name}' execution finished.")

            # Extract result or error message from the tool result object
            result_text = ""
            # Check structure carefully based on actual `session.call_tool` return type
            if (
                hasattr(tool_result, "content")
                and tool_result.content
                and hasattr(tool_result.content[0], "text")
            ):
                result_text = tool_result.content[0].text or ""

            if hasattr(tool_result, "isError") and tool_result.isError:
                error_message = (
                    result_text
                    or f"Tool '{tool_name}' failed without specific error message."
                )
                print(f"  Tool '{tool_name}' reported an error: {error_message}")
                tool_result_payload = {"error": error_message}
            else:
                print(
                    f"  Tool '{tool_name}' succeeded. Result snippet: {result_text[:150]}..."
                )  # Log snippet
                tool_result_payload = {"result": result_text}

        except Exception as e:
            # Catch exceptions during the tool call itself
            error_message = f"Tool execution framework failed: {type(e).__name__}: {e}"
            print(f"  Error executing tool '{tool_name}': {error_message}")
            tool_result_payload = {"error": error_message}

        # Create a FunctionResponse Part to send back to the model
        tool_response_parts.append(
            types.Part.from_function_response(
                name=tool_name, response=tool_result_payload
            )
        )
    print(f"--- Finished executing tool call(s) ---")
    return tool_response_parts


async def run_agent_loop(
    prompt: str,
    client: genai.Client,
    session: ClientSession,
    model_id: str = MODEL_ID,
    max_tool_turns: int = DEFAULT_MAX_TOOL_TURNS,
    initial_temperature: float = DEFAULT_INITIAL_TEMPERATURE,
    tool_call_temperature: float = DEFAULT_TOOL_CALL_TEMPERATURE,
) -> types.GenerateContentResponse:
    """
    Runs a multi-turn conversation loop with a Gemini model, handling tool calls.

    This function orchestrates the interaction between a user prompt, a Gemini
    model capable of function calling, and a session object that provides
    and executes tools. It handles the cycle of:
    1. Sending the user prompt (and conversation history) to the model.
    2. If the model requests tool calls, executing them via the `session`.
    3. Sending the tool execution results back to the model.
    4. Repeating until the model provides a text response or the maximum
       number of tool execution turns is reached.

    Args:
        prompt: The initial user prompt to start the conversation.
        client: An initialized Gemini GenerativeModel client object

        session: An active session object responsible for listing available tools
                 via `list_tools()` and executing them via `call_tool(tool_name, args)`.
                 It's also expected to have an `initialize()` method.
        model_id: The identifier of the Gemini model to use (e.g., "gemini-1.5-pro-latest").
        max_tool_turns: The maximum number of consecutive turns dedicated to tool calls
                        before forcing a final response or exiting.
        initial_temperature: The temperature setting for the first model call.
        tool_call_temperature: The temperature setting for subsequent model calls
                               that occur after tool execution.

    Returns:
        The final Response from the Gemini model after the
        conversation loop concludes (either with a text response or after
        reaching the max tool turns).

    Raises:
        ValueError: If the session object does not provide any tools.
        Exception: Can potentially raise exceptions from the underlying API calls
                   or session tool execution if not caught internally by `_execute_tool_calls`.
    """
    print(
        f"Starting agent loop with model '{model_id}' and prompt: '{prompt[:100]}...'"
    )

    # Initialize conversation history with the user's prompt
    contents: List[types.Content] = [
        types.Content(role="user", parts=[types.Part(text=prompt)])
    ]

    # Ensure the session is ready (if needed)
    if hasattr(session, "initialize") and callable(session.initialize):
        print("Initializing session...")
        await session.initialize()
    else:
        print("Session object does not have an initialize() method, proceeding anyway.")

    # --- 1. Discover Tools from Session ---
    print("Listing tools from session...")
    # Assumes session.list_tools() returns an object with a 'tools' attribute (list)
    # Each item in the list should have 'name', 'description', and 'inputSchema' attributes.
    session_tool_list = await session.list_tools()

    if not session_tool_list or not session_tool_list.tools:
        raise ValueError("No tools provided by the session. Agent loop cannot proceed.")

    # Convert session tools to the format required by the Gemini API
    gemini_tool_config = types.Tool(
        function_declarations=[
            types.FunctionDeclaration(
                name=tool.name,
                description=tool.description,
                parameters=tool.inputSchema,  # Assumes inputSchema is compatible
            )
            for tool in session_tool_list.tools
        ]
    )
    print(
        f"Configured Gemini with {len(gemini_tool_config.function_declarations)} tool(s)."
    )

    # --- 2. Initial Model Call ---
    print("Making initial call to Gemini model...")
    current_temperature = initial_temperature
    response = await client.aio.models.generate_content(
        model=MODEL_ID,
        contents=contents,  # Send updated history
        config=types.GenerateContentConfig(
            temperature=1.0,
            tools=[gemini_tool_config],
        ),  # Keep sending same config
    )
    print("Initial response received.")

    # Append the model's first response (potentially including function calls) to history
    # Need to handle potential lack of candidates or content
    if not response.candidates:
        print("Warning: Initial model response has no candidates.")
        # Decide how to handle this - raise error or return the empty response?
        return response
    contents.append(response.candidates[0].content)

    # --- 3. Tool Calling Loop ---
    turn_count = 0
    # Check specifically for FunctionCall objects in the latest response part
    latest_content = response.candidates[0].content
    has_function_calls = any(part.function_call for part in latest_content.parts)

    while has_function_calls and turn_count < max_tool_turns:
        turn_count += 1
        print(f"\n--- Tool Turn {turn_count}/{max_tool_turns} ---")

        # --- 3.1 Execute Pending Function Calls ---
        function_calls_to_execute = [
            part.function_call for part in latest_content.parts if part.function_call
        ]
        tool_response_parts = await _execute_tool_calls(
            function_calls_to_execute, session
        )

        # --- 3.2 Add Tool Responses to History ---
        # Send back the results for *all* function calls from the previous turn
        contents.append(
            types.Content(role="function", parts=tool_response_parts)
        )  # Use "function" role
        print(f"Added {len(tool_response_parts)} tool response part(s) to history.")

        # --- 3.3 Make Subsequent Model Call with Tool Responses ---
        print("Making subsequent API call to Gemini with tool responses...")
        current_temperature = tool_call_temperature  # Use different temp for follow-up
        response = await client.aio.models.generate_content(
            model=MODEL_ID,
            contents=contents,  # Send updated history
            config=types.GenerateContentConfig(
                temperature=1.0,
                tools=[gemini_tool_config],
            ),
        )
        print("Subsequent response received.")

        # --- 3.4 Append latest model response and check for more calls ---
        if not response.candidates:
            print("Warning: Subsequent model response has no candidates.")
            break  # Exit loop if no candidates are returned
        latest_content = response.candidates[0].content
        contents.append(latest_content)
        has_function_calls = any(part.function_call for part in latest_content.parts)
        if not has_function_calls:
            print(
                "Model response contains text, no further tool calls requested this turn."
            )

    # --- 4. Loop Termination Check ---
    if turn_count >= max_tool_turns and has_function_calls:
        print(
            f"Maximum tool turns ({max_tool_turns}) reached. Exiting loop even though function calls might be pending."
        )
    elif not has_function_calls:
        print("Tool calling loop finished naturally (model provided text response).")

    # --- 5. Return Final Response ---
    print("Agent loop finished. Returning final response.")
    return response

#### Set up MCP client

In [34]:
async def run_simple_agent(server_params, query):
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(
            read,
            write,
        ) as session:
            # Test prompt
            prompt = query
            print(f"Running agent loop with prompt: {prompt}")
            # Run agent loop
            res = await run_agent_loop(prompt, client, session)
            return res

In [35]:
bq_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/bq.py"],
)

In [36]:
med_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/med.py"],
)

In [37]:
nih_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/nih.py"],
)

In [38]:
ct_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/cocktail.py"],
)

In [39]:
bq_query = (
    "Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'"
)
bq_res = await run_simple_agent(bq_server_params, bq_query)
print(bq_res.text)

Running agent loop with prompt: Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'
Starting agent loop with model 'gemini-2.5-flash' and prompt: 'Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'...'
Initializing session...
Listing tools from session...
Configured Gemini with 2 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'list_tables' with args: {'location': 'us', 'project_id': 'dw-genai-dev'}
  Session tool 'list_tables' execution finished.
  Tool 'list_tables' succeeded. Result snippet: No datasets found in project dw-genai-dev....
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further tool calls requested this turn.
Tool calling loop finished naturally

In [40]:
med_query = "Please explain flu in detail."
med_res = await run_simple_agent(med_server_params, med_query)
print(med_res.text)

Running agent loop with prompt: Please explain flu in detail.
Starting agent loop with model 'gemini-2.5-flash' and prompt: 'Please explain flu in detail....'
Initializing session...
Listing tools from session...
Configured Gemini with 1 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'get_medical_term' with args: {'term': 'flu'}
  Session tool 'get_medical_term' execution finished.
  Tool 'get_medical_term' succeeded. Result snippet: What is the <span class="qt0">flu</span>?<p>The <span class="qt0">flu</span>, also called <span class="qt0">influenza</span>, is a respiratory infecti...
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further tool calls requested this turn.
Tool calling loop finished naturally (model 

In [41]:
nih_query = "Please tell me icd-10 code for pneumonia"
nih_res = await run_simple_agent(nih_server_params, nih_query)
print(nih_res.text)

Running agent loop with prompt: Please tell me icd-10 code for pneumonia
Starting agent loop with model 'gemini-2.5-flash' and prompt: 'Please tell me icd-10 code for pneumonia...'
Initializing session...
Listing tools from session...
Configured Gemini with 1 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'get_icd_10_code' with args: {'name_or_code': 'pneumonia'}
  Session tool 'get_icd_10_code' execution finished.
  Tool 'get_icd_10_code' succeeded. Result snippet: No ICD-10 codes found for 'pneumonia'....
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further tool calls requested this turn.
Tool calling loop finished naturally (model provided text response).
Agent loop finished. Returning final response.
I canno

In [42]:
ct_query = "Please tell me the details of cocktail margarita"
bq_res = await run_simple_agent(ct_server_params, ct_query)
print(bq_res.text)

Running agent loop with prompt: Please tell me the details of cocktail margarita
Starting agent loop with model 'gemini-2.5-flash' and prompt: 'Please tell me the details of cocktail margarita...'
Initializing session...
Listing tools from session...
Configured Gemini with 5 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'search_cocktail_by_name' with args: {'name': 'margarita'}
  Session tool 'search_cocktail_by_name' execution finished.
  Tool 'search_cocktail_by_name' succeeded. Result snippet: ID: 11007, Name: Margarita, Category: Ordinary Drink
---
ID: 11118, Name: Blue Margarita, Category: Ordinary Drink
---
ID: 17216, Name: Tommy's Margar...
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further tool calls 

### Option 3 Testing with Google ADK
Note: It works in  Jupyter Notebook only

In [ ]:
!pip install -U google-adk


In [46]:
from google.adk.tools.mcp_tool.mcp_toolset import (
    MCPToolset,
    # SseServerParams,
    StdioServerParameters,
)

ImportError: cannot import name 'SseServerParams' from 'google.adk.tools.mcp_tool.mcp_toolset' (/opt/conda/lib/python3.10/site-packages/google/adk/tools/mcp_tool/mcp_toolset.py)

In [ ]:
import os

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

In [ ]:
from google.adk.tools.mcp_tool.mcp_toolset import (
    MCPToolset,
    # SseServerParams,
    StdioServerParameters,
)
from google.adk.agents.llm_agent import LlmAgent

import asyncio
from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService

load_dotenv()


async def get_tools_async(server_params):
    """Gets tools from MCP Server."""
    tools, exit_stack = await MCPToolset.from_server(connection_params=server_params)
    # MCP requires maintaining a connection to the local MCP Server.
    # Using exit_stack to clean up server connection before exit.
    return tools, exit_stack


async def get_agent_async(server_params):
    """Creates an ADK Agent with tools from MCP Server."""
    tools, exit_stack = await get_tools_async(server_params)
    root_agent = LlmAgent(
        model=MODEL_ID,
        name="ai_assistant",
        instruction="Use tools to get information to answer user questions",
        tools=tools,
    )
    return root_agent, exit_stack

### An Agent built using Google ADK

In [ ]:
async def run_adk_agent(server_params, question):
    session_service = InMemorySessionService()
    artifacts_service = InMemoryArtifactService()
    session = session_service.create_session(state={}, app_name="my_app", user_id="123")

    query = question
    print("[user]: ", query)
    content = types.Content(role="user", parts=[types.Part(text=query)])
    root_agent, exit_stack = await get_agent_async(server_params)
    runner = Runner(
        app_name="my_app",
        agent=root_agent,
        artifact_service=artifacts_service,
        session_service=session_service,
    )
    events_async = runner.run_async(
        session_id=session.id, user_id="123", new_message=content
    )
    # print(events_async)
    response = {}
    async for event in events_async:
        # print(event)
        if event.content.role == "user" and event.content.parts[0].text:
            print("[user]:", event.content.parts[0].text)
            response["user"] = event.content.parts[0].text
        if event.content.parts[0].function_response:
            print("[-tool_response-]", event.content.parts[0].function_response)
            response["-tool_response-"] = event.content.parts[0].function_response
        if event.content.role == "model" and event.content.parts[0].text:
            print("[agent]:", event.content.parts[0].text)
            response["agent"] = event.content.parts[0].text

    await exit_stack.aclose()

    return response

In [ ]:
# For this server, make sure you have Node.js installed on your machine
bnb_server_params = StdioServerParameters(
    command="npx", args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"]
)

In [ ]:
response = await run_adk_agent(
    bnb_server_params,
    "Please find a room in LA, CA, April 15, 2025, checkout date is april 18, 2 adults",
)

In [ ]:
await run_adk_agent(bnb_server_params, "give more details of id 7462294")

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Exiting chat.")
        break
    await run_adk_agent(bnb_server_params, user_input)

In [ ]:
ct_server_params = StdioServerParameters(
    command="python",
    args=["./server/cocktail2.py"],
)

In [ ]:
await run_adk_agent(
    ct_server_params,
    "Please get cocktail margarita id and then full detail of cocktail margarita",
)

### References:

https://modelcontextprotocol.io/introduction  

https://github.com/philschmid/gemini-samples/blob/main/examples/gemini-mcp-example.ipynb  

https://github.com/modelcontextprotocol/python-sdk